In [1]:
from requests.exceptions import ConnectionError, HTTPError, MissingSchema, ReadTimeout
import logging
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta, date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from enum import Enum, IntEnum

In [ ]:
#not important 

def get_all_content_infinite_scroll(scroll_pause_time=1):

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3) #time to load a page
    
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1

    #infinite scroll loop
    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    return soup.main.prettify()


print(get_all_content_infinite_scroll(0.2))

In [ ]:
#not important 

def get_all_article_id():

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3) #time to load a page

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    articles = soup.find_all('article')

    list_of_article_id = [x["id"] for x in articles] 
    
    return list_of_article_id

print(get_all_article_id())

In [ ]:
#not important 

def get_all_article_id_many_subpages(start_page_number=1, end_page_number=3):

    page_number = 1
    list_of_article_id = []
    all_articles = []
    

    try:
        while page_number != end_page_number:

            page_url = f"https://www.pepper.pl/nowe?page={page_number}"

            driver = webdriver.Chrome('./chromedriver') 
            driver.get(page_url) 
            time.sleep(1) #time to load a page

            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')


            articles_from_page = soup.find_all('article')
            #all_articles.append(articles_from_page)
            print(articles_from_page)

            print(len(all_articles))
            print(len(soup))


            for i in articles:
                if i["id"] not in list_of_article_id:
                    list_of_article_id.append(i["id"])
            
            page_number += 1
    except:
        print("Exception occured")

    return list_of_article_id

print(get_all_article_id_many_subpages(1,5))

In [ ]:
#not important 

def get_item_name_like(item):

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3) #time to load a page

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    list_all_items_info = soup.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})

    list_all_items_names = [x.get_text() for x in list_all_items_info]
    
    matches = [x for x in list_all_items_names if item.lower() in x.lower()]

    print(matches)


get_item_name_like("Monitor")

In [ ]:
#not important 

def get_content_item_tag():

    page_url = "https://www.pepper.pl/nowe"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3)

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    all_tags_content = soup.find_all("article")
    
    for i in all_tags_content:
        item_name = i.find_all("cept-tt thread-link linkPlain thread-title--list js-thread-title")
        discount_price = i.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})
        regular_price = i.find_all(attrs={'class': "mute--text text--lineThrough size--all-l size--fromW3-xl"})
        percentage_discount = i.find_all(attrs={'class': "space--ml-1 size--all-l size--fromW3-xl"})
        item_link = i.find_all('a', href=True, text=True)
        
        #(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"}, href=True)
        

        article_id = i["id"]
        print(item_name)
        try:    
            print(float(discount_price[0].get_text().strip('zł').replace('.','').replace(',','.')))
        except: 
            print("N/A")
        try:
            print(float(regular_price[0].get_text().strip('zł').replace('.','').replace(',','.')))
        except:
            print("N/A")
        try:
            print(float(percentage_discount[0].get_text().strip('%')))
        except:
            print("N/A")
        print(item_link[0]['href'])
        print(article_id.strip('thread_'))


    #print(result)




get_content_item_tag()

In [ ]:
#not important 

class Months(Enum):

    sty = '01'
    lut = '02'
    mar = '03'
    kwi = '04'
    maj = '05'
    cze = '06'
    lip = '07'
    sie = '08'
    wrz = '09'
    paz = '10'
    paź = '10'
    lis = '11'
    gru = '12'

    @classmethod
    def to_dict(cls):
        """Returns a dictionary representation of the enum."""
        return {e.name: e.value for e in cls}
    
    @classmethod
    def keys(cls):
        """Returns a list of all the enum keys."""
        return cls._member_names_
    
    @classmethod
    def values(cls):
        """Returns a list of all the enum values."""
        return list(cls._value2member_map_.keys())

def data_conversion(publication_date):

    try:
        if publication_date.endswith(('min', 'g', 's', 'temu')):
            prepared_data = date.today().strftime("%d-%m-%Y")
            return prepared_data
        elif publication_date.startswith(tuple(Months.keys())) and len(publication_date) < 8:      
            if len(publication_date[4:]) == 3:
                day = publication_date[4:6]
            else:
                day = publication_date[4:5].zfill(2)
            month = Months.__members__[publication_date[0:3]].value
            year = str(date.today().year)
            prepared_data = '-'.join([str(day), month, year])
            return prepared_data
        else:
            day = publication_date[4:6]
            month = Months.__members__[publication_date[0:3]].value
            year = publication_date[7:13]
            prepared_data = '-'.join([day, month, year])
            return prepared_data
    except KeyError as e:
        return f'Invalid name of the month: {e}'
    except:
        print('error')



def get_article_data():

    data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

    page_url = "https://www.pepper.pl/nowe?page=25"

    driver = webdriver.Chrome('./chromedriver') 
    driver.get(page_url) 
    time.sleep(3)

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')

    all_tags_content = soup.find_all(lambda tag:tag.name=="article")

    n=1

    for i in all_tags_content:
        publication_date_tag = i.find_all(attrs={'class': "hide--fromW3"})
        item_name = i.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})
        print(f"item number {n}")
        n+=1
        print(item_name[0].get_text()) 

        print(publication_date_tag)
        potentially_data_string_1 = publication_date_tag[0].get_text()
        potentially_data_string_2 = publication_date_tag[1].get_text()
        try:
            formatted_data_1 = data_conversion(potentially_data_string_1)

            if bool(re.search(data_pattern, formatted_data_1)):
                print(data_conversion(potentially_data_string_1) + " string 1 success")
            else:
                raise Exception("didnt work")
        except:
            try: 
                formatted_data_2 = data_conversion(potentially_data_string_2)

                if bool(re.search(data_pattern, formatted_data_2)):
                    print(data_conversion(potentially_data_string_2) + " string 2 success")
                else:
                    raise Exception("didnt work second position")
            except:
                potentially_data_string_3 = publication_date_tag[2].get_text()
                print(data_conversion(potentially_data_string_3) + " string 3 success")
 
       

print(get_article_data())

In [68]:
#important! in the pipeline

action_type = "/nowe?page="
start_page = 25
end_page = 25
website_url = "https://www.pepper.pl"
articles_to_retrieve = 61

#informations = [name, discount_price, regular_price, percentage_discount, link]


class ScrapWebpage:

    def __init__(self, website_url, action_type, articles_to_retrieve, start_page=1):
        self.website_url = website_url
        self.action_type = action_type
        self.articles_to_retrieve = articles_to_retrieve
        self.start_page = start_page


    def scrap_data(self):

        try:
            url_to_scrap = self.website_url + self.action_type + str(self.start_page)
            driver = webdriver.Chrome('./chromedriver') 
            driver.get(url_to_scrap) 
            time.sleep(0.7)
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            return soup
        except ConnectionError as e:
            print(f"ConnectionError occured: {e}. \nTry again later")
        except MissingSchema as e:
            print(f"MissingSchema occured: {e}. \nMake sure that protocol indicator is icluded in the website url")
        except HTTPError as e:
            print(f"HTTPError occured: {e}. \nMake sure that website url is valid")
        except ReadTimeout as e:
            print(f"ReadTimeout occured: {e}. \nTry again later")


    def infinite_scroll_handling(self):

        try:
            flag = True
            retrived_articles = list()

            while flag:
                soup = self.scrap_data()
                articles = soup.find_all('article')
                retrived_articles += articles

                if len(retrived_articles) >= self.articles_to_retrieve:
                    flag = False

                self.start_page += 1

            return retrived_articles[:self.articles_to_retrieve]
        except IndexError as e:
            raise IndexError("There aren't that many articles, try retrieve lower quantity of articles")

        
    
    def choose_informations_to_scrap(self):
        
        """info_call = {artice_id:         ["id"],
                name:                   attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"},
                date:                   ItemAddedDataRetrieve(),
                discount_price:         attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"},
                regular_price:          attrs={'class': "mute--text text--lineThrough size--all-l size--fromW3-xl"},
                percentage_discount:    attrs={'class': "space--ml-1 size--all-l size--fromW3-xl"},
                link:                   'a', href=True, text=True}"""

        retrived_articles = self.infinite_scroll_handling()

        for article in retrived_articles:
            print(GetItemName(article).get_data())

            """for info in self.informations:
                #info_call.get(info)
                info = article.find_all(attrs={info_call.get(info))
                print(info)
"""






            """item_name = i.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})
        discount_price = i.find_all(attrs={'class': "thread-price text--b cept-tp size--all-l size--fromW3-xl"})
        regular_price = i.find_all(attrs={'class': "mute--text text--lineThrough size--all-l size--fromW3-xl"})
        percentage_discount = i.find_all(attrs={'class': "space--ml-1 size--all-l size--fromW3-xl"})
        item_link = i.find_all('a', href=True, text=True)"""
        
        

        #(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"}, href=True)
        







    

    

output = ScrapWebpage(website_url, action_type, articles_to_retrieve)
output.choose_informations_to_scrap()


/tmp/ipykernel_278/3414485110.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


Deadly Premonition: The Director's Cut za 8,99 zł i Deadly Premonition: The Director's Cut - Deluxe Edition za 10,79 zł @ Steam
Elektryczny śrubokręt FERM 4V
Ręcznik papierowy Kartika XMASS 3 warstwy 2 rolki za 4,99zł w Biedronce w Opolu
Piwo Browar Jana - Peach Gose i (Non)Pastry Sour - Market Point Kraków al. Pokoju 78
Ładowarka samochodowa micro USB Titanum
Firewatch za 11,90 zł z Węgierskiego Xbox Store @ Xbox One / Xbox Series / PC
PS Plus Celebration Avatary + tapety za darmo z kodem @ PS Store
Kurtka motocyklowa RST tracktech evo 4 rozmiar S, M 748zł
Snow Bros. Nick & Tom Special Nintendo Switch
Laptop 3D ACER Predator Helios 300 PH315-55S SpatialLabs 15.6" IPS i9-12900H 32GB RAM 2 x 1TB SSD GeForce RTX3080 Windows 11 Home
Samsung Galaxy Watch 5 Pro (R920) możliwe 999zł (500zł zwrot cashback)
INDIANA JONES AND THE LAST CRUSADE PC @ Steam
Laptop ACER Nitro 5 AN515-58 15.6" IPS 165Hz i7-12700H 16GB RAM 512GB SSD GeForce RTX3070Ti
Laptop ASUS TUF Gaming A15 FA506QM-HN008W 15.6" IPS

In [66]:
#important! in the pipeline


class GetItemName:

    def __init__(self, article):
        self.article = article
    
    def get_data(self):
        try:
            name = self.article.find_all(attrs={'class': "cept-tt thread-link linkPlain thread-title--list js-thread-title"})
            name = name[0].get_text()
            return name
        except TypeError as e:
            raise TypeError(f"Invalid html class name: {e}")

        


In [ ]:
#important! in the pipeline


class Months(Enum):

    sty = '01'
    lut = '02'
    mar = '03'
    kwi = '04'
    maj = '05'
    cze = '06'
    lip = '07'
    sie = '08'
    wrz = '09'
    paz = '10'
    paź = '10'
    lis = '11'
    gru = '12'

    @classmethod
    def to_dict(cls):
        """Returns a dictionary representation of the enum."""
        return {e.name: e.value for e in cls}
    
    @classmethod
    def keys(cls):
        """Returns a list of all the enum keys."""
        return cls._member_names_
    
    @classmethod
    def values(cls):
        """Returns a list of all the enum values."""
        return list(cls._value2member_map_.keys())

class GetItemAddedDate(Months):

    def data_format_conversion(self, scraped_publication_date):

        old_dates_data_pattern = "[A-Za-z]+\s\d\d\.\s[0-9]+"

        try:
            if scraped_publication_date.endswith(('min', 'g', 's', 'temu')):
                prepared_data = date.today().strftime("%d-%m-%Y")
                return prepared_data
            elif scraped_publication_date.startswith(tuple(Months.keys())) and len(scraped_publication_date) < 8:      
                if len(scraped_publication_date[4:]) == 3:
                    day = scraped_publication_date[4:6]
                else:
                    day = scraped_publication_date[4:5].zfill(2)
                month = Months.__members__[scraped_publication_date[0:3]].value
                year = str(date.today().year)
                prepared_data = '-'.join([str(day), month, year])
                return prepared_data
            elif bool(re.search(old_dates_data_pattern, scraped_publication_date)):
                day = scraped_publication_date[4:6]
                month = Months.__members__[scraped_publication_date[0:3]].value
                year = scraped_publication_date[8:13]
                prepared_data = '-'.join([day, month, year])
                return prepared_data
        except KeyError as error:
            raise KeyError(f"Invalid name of the month {error}")


    def first_index_position_data_searching(self, html_tag_with_publication_data):

        output_data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

        string_to_check = html_tag_with_publication_data[0].get_text()

        try:
            formatted_data = self.data_format_conversion(string_to_check)
            if bool(re.search(output_data_pattern, formatted_data)):
                return f"{data_conversion(string_to_check)} + ' string 1 success'"
        except Exception as error:
            print(error)

    def second_index_position_data_searching(self, html_tag_with_publication_data):
        
        output_data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

        string_to_check = html_tag_with_publication_data[1].get_text()

        try:
            formatted_data = self.data_format_conversion(string_to_check)
            if bool(re.search(output_data_pattern, formatted_data)):
                return f"{data_conversion(string_to_check)} + ' string 2 success'"
        except Exception as error:
            print(error)

    def third_index_position_data_searching(self, html_tag_with_publication_data):
        
        output_data_pattern = "\d{2}[/.-]\d{2}[/.-]\d{4}"

        string_to_check = html_tag_with_publication_data[2].get_text()

        try:
            formatted_data = self.data_format_conversion(string_to_check)
            if bool(re.search(output_data_pattern, formatted_data)):
                return f"{data_conversion(string_to_check)} + ' string 3 success'"
        except Exception as error:
            print(error)

    def scrap_page_and_receive_data

